# *Resultados Fundamentais - Extração de Dados Financeiros e de Empresas*
*Este script extrai dados financeiros de uma tabela na página de resultados do site "[Fundamentus](https://www.fundamentus.com.br)" e informações sobre papeis e empresas de outra tabela na página de detalhes do mesmo site.*

## *Instalação das Bibliotecas Necessárias*

In [1]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## *Importação das Bibliotecas Necessárias*

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException
import pandas as pd
import os, time

##  *Configuração e Inicialização do WebDriver do Selenium*

In [3]:
# Configuração e Inicialização do WebDriver do Selenium em Modo Headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Executar o Chrome em modo headless (sem interface gráfica)
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), "balancos"),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

## *URLs das Páginas a Serem Acessadas*

In [4]:
url_pagina1 = 'https://www.fundamentus.com.br/resultado.php'
url_pagina2 = 'https://www.fundamentus.com.br/detalhes.php'

## *Localização e Extração das Tabelas* 

In [5]:
# Acessando a Página 1 (Resultados)
driver.get(url_pagina1)

# Localizando e Extraindo a Primeira Tabela (Dados Financeiros)
local_tabela = '/html/body/div[1]/div[2]/table'
elemento = driver.find_element("xpath", local_tabela)

# Obter o HTML da tabela
html_tabela = elemento.get_attribute('outerHTML')

# Usando StringIO para ler o HTML
html_string_io = StringIO(html_tabela)

In [6]:
# Acessando a Página 2 (Detalhes de Empresas)
driver.get(url_pagina2)

# Aguardar até que a página esteja completamente carregada
driver.implicitly_wait(10)

# Localizando e Extraindo a Segunda Tabela (Papeis e Nomes de Empresas)
local_tabela_2 = '/html/body/div[1]/div[2]/div[1]/div/table'

# Tentar localizar o elemento novamente
try:
    elemento_2 = driver.find_element("xpath", local_tabela_2)
    # Obter o HTML da tabela
    html_tabela_2 = elemento_2.get_attribute('outerHTML')
except StaleElementReferenceException:
    # Se ocorrer um erro StaleElementReferenceException, tente localizar o elemento novamente
    elemento_2 = driver.find_element("xpath", local_tabela_2)
    # Obter o HTML da tabela
    html_tabela_2 = elemento_2.get_attribute('outerHTML')

# Usando StringIO para ler o HTML
html_string_io_2 = StringIO(html_tabela_2)

## Conversão em DataFrame Pandas

In [7]:
df_financas = pd.read_html(html_string_io, decimal=',', thousands='.')[0]

df_empresas = pd.read_html(html_string_io_2)[0]

# Extraindo o Papel e o Nome da Empresa, e Removendo a Coluna de Razão Social
df_empresas.columns = ['Papel', 'Nome', 'Razão Social']
df_empresas.drop(columns=['Razão Social'], inplace=True)


## Exibindo os DataFrames

In [8]:
## Tabela com os dados financeiros
print("Tabela com os dados financeiros:")
display(df_financas)

Tabela com os dados financeiros:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,10.17,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
1,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
2,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
3,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
4,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,DMFN3,14.00,862.53,60.25,0.000,"0,04%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","6,99%",781.4,1.435900e+07,0.00,"34,42%"
983,CSNA3,13.29,953.20,1.02,0.402,"10,98%",0.192,2.39,2.92,-0.45,...,5.01,"13,77%","1,70%",1.29,"8,82%","0,11%",91657400.0,1.729740e+10,2.68,"14,14%"
984,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
985,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


In [9]:
print("\nTabela com os papeis e seus nomes:")
display(df_empresas)


Tabela com os papeis e seus nomes:


,Papel,Nome
0,AALR3,ALLIAR
1,ABCB3,ABC Brasil
2,ABCB4,ABC Brasil
3,ABEV3,AMBEV S/A
4,ABRE3,SOMOS EDUCA
...,...,...
1029,WMBY3,WEMBLEY SOCIEDADE ANÔNIMA
1030,WSON33,Wilson Sons
1031,YBRA4,OPPORTUNITY ALEF S/A
1032,YDUQ3,YDUQS PART


## Balanços

### Criando a pasta 'balancos' se não existir

In [23]:
if not os.path.exists('balancos'):
    os.makedirs('balancos')

### DataFrame para armazenar papéis com erro no download

In [24]:
df_erros = pd.DataFrame(columns=['Papel', 'Erro'])

### DataFrame para armazenar os arquivos baixados

In [25]:
df_baixados = pd.DataFrame(columns=['Papel', 'Nome_Arquivo'])

###  *Configuração e Inicialização do WebDriver do Selenium*

In [58]:
# Configuração e Inicialização do WebDriver do Selenium em Modo Headless
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), "balancos"),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

### Acessando a página de cada papel em df_empresas['Papel'] e baixando o arquivo

In [59]:
for papel in df_empresas['Papel']:
    url_papeis = f'https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile'
    driver.get(url_papeis)
    driver.implicitly_wait(10)
    
    try:
        # Localizando e clicando no botão de download
        download_button = driver.find_element("css selector", "a.bt-baixar")
        download_button.click()
        
        # Aguardando o download completar
        time.sleep(5)  # Ajuste o tempo conforme necessário
        
        # Verificando se há arquivos no diretório
        download_path = os.path.join(os.getcwd(), "balancos")
        downloaded_files = [f for f in os.listdir(download_path) if f.endswith('.zip')]
        
        if downloaded_files:
            # Registrando o papel baixado com sucesso
            df_baixados = pd.concat([df_baixados, pd.DataFrame({'Papel': [papel]})])
            
            # Print indicando que o arquivo foi baixado com sucesso
            print(f"Arquivo para {papel} baixado com sucesso.")
        else:
            raise ValueError("Nenhum arquivo foi baixado.")
            
    except (NoSuchElementException, TimeoutException, Exception) as e:
        print(f"Erro ao baixar o arquivo para {papel}: {e}")
        # Adicionando o papel e o erro ao DataFrame de erros
        df_erros = pd.concat([df_erros, pd.DataFrame({'Papel': [papel], 'Erro': [str(e)]})])

Arquivo para AALR3 baixado com sucesso.
Arquivo para ABCB3 baixado com sucesso.
Arquivo para ABCB4 baixado com sucesso.
Arquivo para ABEV3 baixado com sucesso.
Erro ao baixar o arquivo para ABRE3: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.bt-baixar"}
  (Session info: chrome=124.0.6367.210); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00AAC113+48259]
	(No symbol) [0x00A3CA41]
	(No symbol) [0x00930A17]
	(No symbol) [0x00970BED]
	(No symbol) [0x00970C9B]
	(No symbol) [0x009ABC12]
	(No symbol) [0x00990DE4]
	(No symbol) [0x009A9B9C]
	(No symbol) [0x00990B36]
	(No symbol) [0x0096570D]
	(No symbol) [0x009662CD]
	GetHandleVerifier [0x00D665A3+2908435]
	GetHandleVerifier [0x00DA3BBB+3159851]
	GetHandleVerifier [0x00B450CB+674875]
	GetHandleVerifier [0x00B4B28C+699900]
	(No symbol) [0x00A46244]
	(No symbol) [0x00A4

### Renomeação dos arquivos baixados

In [62]:
download_path = os.path.join(os.getcwd(), "balancos")
for index, row in df_baixados.iterrows():
    try:
        # Esperar até que o arquivo esteja disponível para renomeação
        while not any(file.startswith(row['Papel']) for file in os.listdir(download_path)):
            time.sleep(1)
        
        # Encontrar o arquivo e renomeá-lo
        old_file = next(file for file in os.listdir(download_path) if file.startswith(row['Papel']))
        new_file_name = f"{row['Papel']}.zip"
        os.rename(os.path.join(download_path, old_file), os.path.join(download_path, new_file_name))
        
        # Atualizar o DataFrame de arquivos baixados com o nome do arquivo
        df_baixados.at[index, 'Nome_Arquivo'] = new_file_name
    except Exception as e:
        print(f"Erro ao renomear o arquivo para {row['Papel']}: {e}")

# Atualizar o DataFrame de erros removendo os papéis que foram baixados com sucesso
df_erros = df_erros[~df_erros['Papel'].isin(df_baixados['Papel'])]

KeyboardInterrupt: 

### Exibindo os erros de download

In [63]:
papeis_baixados = df_baixados['Papel'].tolist()
df_erros = df_erros[~df_erros['Papel'].isin(papeis_baixados)]

# Exibindo os erros de download
print("\nPapeis com erro no download:")
print(df_erros)


Papeis com erro no download:
    Papel                                               Erro
0   ABRE3  Message: no such element: Unable to locate ele...
0   ABRE3  'WebDriver' object has no attribute 'find_elem...
0   ABRE3  'WebDriver' object has no attribute 'find_elem...
0   ABRE3  'WebDriver' object has no attribute 'find_elem...
0   ABRE3  Message: no such element: Unable to locate ele...
0   ABRE3  Message: no such element: Unable to locate ele...
0   ALSO3  Message: no such element: Unable to locate ele...
0   AORE3  Message: no such element: Unable to locate ele...
0  AURA32  Message: no such element: Unable to locate ele...
0   BBRK3  Message: no such element: Unable to locate ele...
0   BKBR3  Message: no such element: Unable to locate ele...
0   BPNM4  Message: no such element: Unable to locate ele...
0   BRDT3  Message: no such element: Unable to locate ele...
0   BRIN3  Message: no such element: Unable to locate ele...
0   BTOW3  Message: no such element: Unable to locate e

In [64]:
print("\nPapeis Baixados:")
print(df_baixados)


Papeis Baixados:
     Papel Nome_Arquivo
0    ABYA3          NaN
0    AALR3          NaN
0    AALR3          NaN
0    ABCB3          NaN
0    ABCB4          NaN
..     ...          ...
0    WLMM4          NaN
0    WMBY3          NaN
0   WSON33          NaN
0    YDUQ3          NaN
0    ZAMP3          NaN

[992 rows x 2 columns]


## Fechando o Driver Selenium

In [54]:
# Fechar o WebDriver do Selenium após concluir a extração e exibição dos dados
driver.quit()